# Git Commit Scripts

In [1]:
import os
import subprocess

## Prendo informazioni dai files

Funzioni

In [2]:
# Private functions
def _get_git_repo_root(start_path: str) -> str:
    """
    Trova la root della git repository a partire da una directory.
    """
    try:
        result = subprocess.run(
            ['git', 'rev-parse', '--show-toplevel'],
            cwd=start_path,
            stdout=subprocess.PIPE,
            stderr=subprocess.DEVNULL,
            check=True,
            text=True
        )
        return result.stdout.strip()
    except subprocess.CalledProcessError:
        return None

# Public functions

def get_big_files(path:str, paths_to_ignore:list[str] = ["../.git"]) -> list[dict] | None:
    """
    Analizza tutti i file contenuti nella directory ricorsivamente.
    Restituisce in una lista tutti i file superiori a 100mb composta da dizionari:
      {path: str, size: float(in Mb)}
    ignore_paths >> Può essere fornita una lista di path da ignorare dal controllo
    """
    big_files = []
    for root, dirs, files in os.walk(path):

        # Controlla se è un path da ignorare
        if any(path in root for path in paths_to_ignore):
            continue

        for file in files:
            try:
                file_path = os.path.join(root, file)
                size = os.path.getsize(file_path)
                if size > 102760448: # 98Mb ci teniamo larghi
                    mb_size = size/1048576
                    big_files.append({"name": file, "path": file_path, "size": mb_size})
            except Exception as e:
                print(f"Errore con il file {file_path}: {e}")
    return big_files

def is_tracked_from_git_repo(path:str) -> bool:
    """
    Ritorna True se il file è trackato da git, False altrimenti.
    """
    # Prende la directory in cui si trova il file
    file_dir = os.path.dirname(path)

    repo_root = _get_git_repo_root(file_dir)
    if not repo_root:
        return False  # Non è in una repo Git

    try:
        # Esegue il comando git ls-files sul file specificato
        rel_path = os.path.relpath(path, repo_root)
        subprocess.run(
            ['git', 'ls-files', '--error-unmatch', rel_path],
            cwd=file_dir,  # Esegue il comando nella directory del file
            stdout=subprocess.DEVNULL,
            stderr=subprocess.DEVNULL,
            check=True
        )
        return True
    except subprocess.CalledProcessError:
        return False
    except Exception as e:
        print(f"Errore durante il controllo git per {path}: {e}")
        return False

def is_in_gitignore(path_file: str) -> bool:
    abs_path = os.path.abspath(path_file)
    file_dir = os.path.dirname(abs_path)
    repo_root = _get_git_repo_root(file_dir)
    
    if not repo_root:
        return False  # Non siamo in una repo Git

    # Path relativo alla root della repo
    rel_path = os.path.relpath(abs_path, repo_root)

    try:
        subprocess.run(
            ['git', 'check-ignore', rel_path],
            cwd=repo_root,
            stdout=subprocess.DEVNULL,
            stderr=subprocess.DEVNULL,
            check=True
        )
        return True
    except subprocess.CalledProcessError:
        return False

Main codice

In [3]:
big_files = get_big_files("../")

for big_file in big_files:
    big_file["tracked"] = is_tracked_from_git_repo(big_file["path"])
    big_file["gitignored"] = is_in_gitignore(big_file["path"])

    # Un file >100MB è committabile solo se NON è tracciato e IGNORATO da Git
    big_file["committable"] = not big_file["tracked"] and big_file["gitignored"]     

## Test commit

**True** = Posso committare in tranquillità  
**False** = C'è qualche file che ti bloccherà tutto dopo il suo commit

In [4]:
all(big_file["committable"] == True for big_file in big_files)

True

## Tenta la correzzione automatica **prima del disastro**

Funzioni

In [162]:
def append_to_gitignore_if_needed(path: str, gitignore_path: str):
    """
    ../Data\Clean\Rete_ciclabile_stradale\rete_ciclabile_completa.geojson
    """
    path = path.replace("\\", "/").strip(".").strip("/")
    try:
        with open(gitignore_path, "r") as f:
            lines = f.read().splitlines()
    except FileNotFoundError:
        lines = []

    if path not in lines:
        with open(gitignore_path, "a") as f:
            f.write(f"\n{path}")

<>:2: SyntaxWarning: invalid escape sequence '\C'
<>:2: SyntaxWarning: invalid escape sequence '\C'
C:\Users\aless\AppData\Local\Temp\ipykernel_27640\2423587667.py:2: SyntaxWarning: invalid escape sequence '\C'
  """


Rimuoviamo dal tracking se presenti

In [163]:
repo_root = _get_git_repo_root("./")
for big_file in big_files:
    if not big_file["committable"]:
        if big_file["tracked"]:
            # Rimuovi dal tracking (senza cancellarlo dal disco)
            subprocess.run(["git", "rm", "--cached", big_file["path"]], cwd=repo_root)

        append_to_gitignore_if_needed(big_file["path"], os.path.join(repo_root, ".gitignore"))

subprocess.run(["git", "add", ".gitignore"], cwd=repo_root)
subprocess.run(["git", "commit", "-m", "Fix: removed >100MB files from tracking and updated .gitignore"], cwd=repo_root)

CompletedProcess(args=['git', 'commit', '-m', 'Fix: removed >100MB files from tracking and updated .gitignore'], returncode=0)